In [128]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd

In [4]:
# raceMap = {'JMJE001': 'all', 'JMJE012': 'latino', 'JMJE003': 'white', 'JMJE004': 'black', 
#            'JMJE005': 'native', 'JMJE006': 'asian', 'JMJE007': 'hawaiian_other_pacific'}
# raceMap = {'E001': 'all', 'E012': 'latino', 'E003': 'white', 'E004': 'black', 
#            'E005': 'native', 'E006': 'asian', 'E007': 'hwPac'}
# raceKeys = list(raceMap.keys())

In [5]:
# raceColumnSuffixes = ['E001', 'E012', 'E003', 'E004', 'E005', 'E006', 'E007']

In [134]:
fips = ['01', '04', '05', '06', '08', '09', '10', 
        '11', '12', '13', '16' ,'17', '18', '19', 
        '20', '21', '22', '23', '24', '25', '26',
        '27', '28', '29', '30', '31', '32', '33', 
        '34', '35', '36', '37', '38', '39', '40', 
        '41', '42', '44', '45', '46', '47', '48', 
        '49', '50', '51', '53', '54', '55', '56']

Import ACS data (2007-2014)

In [224]:
raceMap = {'E001': 'all', 'E012': 'latino', 'E003': 'white', 'E004': 'black', 
           'E005': 'native', 'E006': 'asian', 'E007': 'hwPac'}
raceKeys = list(raceMap.keys())

popData07_14 = pd.DataFrame()

dataPath = 'raw/acs'
for filename in os.listdir(dataPath):
    if filename.endswith('.csv'):
        print(filename)
        
        nhgisCode=''
        with open(os.path.join(dataPath, filename[:-4] + '_codebook.txt'), 'r') as codebook:
            cb = codebook.read().replace('\n', '')
            nhgisCode = cb[cb.find('E001') - 4: cb.find('E001')].strip()
            
        columns = [nhgisCode + s for s in raceKeys]
        
        df = pd.read_csv(os.path.join(dataPath, filename), encoding = "ISO-8859-1")
        df = df[['GISJOIN', 'YEAR', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA'] + columns]
        df.COUNTYA = df.COUNTYA.astype(str)
        df.STATEA = df.STATEA.astype(str)
        df.COUNTYA = np.where(df.COUNTYA.str.len() == 1, '00' + df.COUNTYA, df.COUNTYA)
        df.COUNTYA = np.where(df.COUNTYA.str.len() == 2, '0' + df.COUNTYA, df.COUNTYA)
        df.STATEA = np.where(df.STATEA.str.len() == 1, '0' + df.STATEA, df.STATEA)
        df['countyCode'] = df.STATEA + df.COUNTYA
        df['year'] = df.YEAR.str.slice(0,4).astype(int) + 2
        df['state'] = df.STATE
        
        df = pd.melt(df, id_vars=['GISJOIN', 'state', 'year'], value_vars=columns)
        df.rename(columns={'value': 'pop', 'variable': 'race'}, inplace=True)
        df.race = df.race.str.slice(-4)
        df.race = df.race.replace(raceMap)
        
        popData07_14 = popData07_14.append(df)

nhgis0027_ds195_20095_2009_blck_grp.csv
nhgis0027_ds206_20145_2014_blck_grp.csv
nhgis0027_ds176_20105_2010_blck_grp.csv
nhgis0027_ds225_20165_2016_blck_grp.csv
nhgis0027_ds215_20155_2015_blck_grp.csv
nhgis0027_ds184_20115_2011_blck_grp.csv
nhgis0027_ds201_20135_2013_blck_grp.csv


In [225]:
popData07_14 = popData07_14.loc[~popData07_14.state.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]

popData07_14 = pd.pivot_table(popData07_14, values = 'pop', 
                              index=['GISJOIN', 'year'], 
                              columns = 'race').reset_index()

Import Decenial Census data (2000/2010)

In [226]:
# 2000
df = pd.read_csv('raw/census/nhgis0028_ds147_2000_blck_grp.csv', encoding = "ISO-8859-1")

raceMap = {'FYF001': 'white', 'FYF002': 'black', 'FYF003': 'native', 'FYF004': 'asian', 
           'FYF005': 'hwPac', 'FYF006': 'otherNonLat', 'FYF007': 'twoNonLat', 
           'FYF008': 'lat_white', 'FYF009': 'lat_black', 'FYF010': 'lat_native', 
           'FYF011': 'lat_asian', 'FYF012': 'lat_hwPac', 'FYF013': 'lat_other', 'FYF014':'lat_two'}
raceKeys = list(raceMap.keys())

df = df[['GISJOIN', 'YEAR', 'STATE'] + raceKeys]
df['latino'] = df.FYF008 + df.FYF009 + df.FYF010 + df.FYF011 + df.FYF012 + df.FYF013 + df.FYF014
df['all'] = df.FYF001 + df.FYF002 + df.FYF003 + df.FYF004 + df.FYF005 + df.FYF006 + df.FYF007 + \
            df.FYF008 + df.FYF009 + df.FYF010 + df.FYF011 + df.FYF012 + df.FYF013 + df.FYF014
df.drop(['FYF006', 'FYF007','FYF008', 'FYF009', 'FYF010', 
         'FYF011', 'FYF012', 'FYF013', 'FYF014'], axis=1, inplace=True)
df['year'] = df.YEAR
df['state'] = df.STATE

popData2000 = pd.melt(df, id_vars=['GISJOIN', 'state', 'year'], value_vars=['all', 'latino', 'FYF001', 'FYF002', 
                                                                   'FYF003', 'FYF004', 'FYF005'])
popData2000.rename(columns={'value': 'pop', 'variable': 'race'}, inplace=True)
popData2000.race = popData2000.race.replace(raceMap)
popData2000 = popData2000.loc[~popData2000.state.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]
# popData2000.replace({'state': {'District Of Columbia': 'District of Columbia'}}, inplace=True)

popData2000 = pd.pivot_table(popData2000, values = 'pop', 
                              index=['GISJOIN', 'year'], 
                              columns = 'race').reset_index()

In [227]:
# 2010
df = pd.read_csv('raw/census/nhgis0028_ds172_2010_blck_grp.csv', encoding = "ISO-8859-1")

raceMap = {'H7Z001':'all', 'H7Z010': 'latino', 'H7Z003': 'white', 'H7Z004': 'black', 
           'H7Z005': 'native', 'H7Z006': 'asian', 'H7Z007': 'hwPac'}
raceKeys = list(raceMap.keys())

df = df[['GISJOIN', 'YEAR', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA'] + raceKeys]
df['year'] = df.YEAR
df['state'] = df.STATE

popData2010 = pd.melt(df, id_vars=['GISJOIN', 'state', 'year'], value_vars=raceMap)
popData2010.rename(columns={'value': 'pop', 'variable': 'race'}, inplace=True)
popData2010.race = popData2010.race.replace(raceMap)
popData2010 = popData2010.loc[~popData2010.state.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]
# popData2010.replace({'state': {'District Of Columbia': 'District of Columbia'}}, inplace=True)

popData2010 = pd.pivot_table(popData2010, values = 'pop', 
                              index=['GISJOIN', 'year'], 
                              columns = 'race').reset_index()

In [233]:
len(popData2010)

216331

In [229]:
popData = popData2000.append(popData2010)
popData = popData.append(popData07_14)

In [231]:
years = list(popData.year.unique())

In [249]:
years = [2008, 2009, 2011, 2012, 2013, 2014]

In [251]:
for y in years:
    popDataSubset = popData.loc[popData.year == y]

    if y < 2008:
        fp = "shapefiles/US_blck_grp_2000.shp"
        data = gpd.read_file(fp)
        data = data.loc[data.FIPSSTCO.str.slice(0,2).isin(fips)]
    elif y == 2008:
        fp = "shapefiles/US_blck_grp_2010.shp"
        data = gpd.read_file(fp)
        data = data.loc[data.STATEFP10.isin(fips)]         
    elif y == 2010:
        fp = "shapefiles/US_blck_grp_2010.shp"
        data = gpd.read_file(fp)
        data = data.loc[data.STATEFP10.isin(fips)] 
    else:
        fp = "shapefiles/US_blck_grp_" + str(y+2) + ".shp"
        data = gpd.read_file(fp)
        data = data.loc[data.STATEFP.isin(fips)] 

    data = data[['GISJOIN', 'geometry']]
    data = data.merge(popDataSubset, how='outer', on='GISJOIN', indicator=True)
    if sum(data.loc[data._merge == 'right_only']['all']) > 300000:
        print("More than 300,000 people are being dropped in merge")
        break
    elif len(data) > 250000:
        print("Bad merge: too many blocks")
        break
    else:
        data = data.loc[data._merge != 'right_only']
        data.drop(['_merge'], axis=1, inplace=True)
        s = {'geometry': 'Polygon', 'properties': {'GISJOIN': 'str:20',
                                                   'year': 'int:4', 
                                                   'all': 'int', 
                                                   'asian': 'int', 
                                                   'black': 'int',
                                                   'hwPac': 'int',
                                                   'latino': 'int', 
                                                   'native': 'int', 
                                                   'white': 'int'}}
        outfp = r"shapefiles/pop_" + str(y) + ".shp"
        data.to_file(outfp, schema=s)
        print(outfp)

shapefiles/pop_2008.shp
shapefiles/pop_2009.shp
shapefiles/pop_2011.shp
shapefiles/pop_2012.shp
shapefiles/pop_2013.shp
shapefiles/pop_2014.shp


In [242]:
popDataSubset = popData.loc[popData.year == 2008]

In [243]:
popDataSubset

race,GISJOIN,year,all,asian,black,hwPac,latino,native,white
1,G01000100201001,2008,547,0,25,0,15,0,507
8,G01000100201002,2008,1262,0,268,0,0,0,917
15,G01000100202001,2008,1313,42,771,0,6,0,494
22,G01000100202002,2008,707,0,424,0,0,0,283
29,G01000100203001,2008,2825,10,338,0,39,0,2438
36,G01000100203002,2008,718,0,250,0,0,0,468
43,G01000100204001,2008,1310,0,0,0,24,0,1281
50,G01000100204002,2008,2022,19,109,0,64,0,1816
57,G01000100204003,2008,972,0,3,0,23,0,927
64,G01000100204004,2008,536,0,0,0,17,0,519


In [244]:
len(popDataSubset)

216331

In [213]:
fp = "shapefiles/US_blck_grp_2000.shp"
data2000 = gpd.read_file(fp)
data2000 = data2000.loc[data2000.FIPSSTCO.str.slice(0,2).isin(fips)]
# data = data.loc[data.STATEFP.isin(fips)]
# data2010 = data2010.loc[data2010.STATEFP10.isin(fips)] 

AttributeError: 'GeoDataFrame' object has no attribute 'FIPSSTCO'

In [245]:
# data = data[['GISJOIN', 'geometry']]
data_merged = data2010.merge(popDataSubset, how='outer', on='GISJOIN', indicator=True)

In [246]:
len(data_merged)

216331

In [247]:
sum(data_merged.loc[data_merged._merge == 'right_only']['all'])

137

In [133]:
data

,GISJOIN,geometry,state,year,all,asian,black,hwPac,latino,native,white
0,G02001300001001,"(POLYGON ((-4346585.491069954 3487194.2971585,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G02001300001002,(POLYGON ((-4123589.88224305 3385462.503652033...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G02001300001003,(POLYGON ((-3976134.477084737 3293346.26874719...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G02001600001001,(POLYGON ((-5208962.656848538 3967672.91484846...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G02001600002001,(POLYGON ((-4401413.492615318 3526122.50888137...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,G02001600002002,(POLYGON ((-4448851.039372857 3506061.18050186...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,G02002000001011,(POLYGON ((-3051360.562427869 3474454.60018076...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,G02002000001012,"POLYGON ((-3049674.8560571 3469700.52036743, -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,G02002000001013,POLYGON ((-3056371.078256964 3472596.726584403...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,G02002000001021,"POLYGON ((-3060157.444489984 3472529.38248052,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


END

In [39]:
county2GISJOIN = popData[['GISJOIN', 'countyCode']].drop_duplicates()

In [40]:
pd.to_pickle(county2GISJOIN, path='county2GISJOIN.pkl')

In [42]:
popData.loc[popData.countyCode == '46102']

race,GISJOIN,countyCode,COUNTY,STATE,year,all,asian,black,hwPac,latino,native,white
1423931,G46010209405001,46102,Oglala Lakota County,South Dakota,2013,1336,0,0,0,23,1221,84
1423932,G46010209405001,46102,Oglala Lakota County,South Dakota,2014,1428,0,0,0,51,1267,101
1423933,G46010209405002,46102,Oglala Lakota County,South Dakota,2013,1459,0,0,0,22,1377,43
1423934,G46010209405002,46102,Oglala Lakota County,South Dakota,2014,1636,0,0,0,89,1493,39
1423935,G46010209405003,46102,Oglala Lakota County,South Dakota,2013,2008,0,51,0,8,1892,57
1423936,G46010209405003,46102,Oglala Lakota County,South Dakota,2014,1855,0,17,0,27,1785,26
1423937,G46010209408001,46102,Oglala Lakota County,South Dakota,2013,746,0,0,0,0,597,43
1423938,G46010209408001,46102,Oglala Lakota County,South Dakota,2014,724,0,0,0,0,567,20
1423939,G46010209408002,46102,Oglala Lakota County,South Dakota,2013,1513,0,17,0,0,1384,112
1423940,G46010209408002,46102,Oglala Lakota County,South Dakota,2014,1630,0,17,0,0,1498,115


In [27]:
missing = ['02158',
 '46102',
 '72001',
 '72003',
 '72005',
 '72007',
 '72009',
 '72011',
 '72013',
 '72015',
 '72017',
 '72019',
 '72021',
 '72023',
 '72025',
 '72027',
 '72029',
 '72031',
 '72033',
 '72035',
 '72037',
 '72039',
 '72041',
 '72043',
 '72045',
 '72047',
 '72049',
 '72051',
 '72053',
 '72054',
 '72055',
 '72057',
 '72059',
 '72061',
 '72063',
 '72065',
 '72067',
 '72069',
 '72071',
 '72073',
 '72075',
 '72077',
 '72079',
 '72081',
 '72083',
 '72085',
 '72087',
 '72089',
 '72091',
 '72093',
 '72095',
 '72097',
 '72099',
 '72101',
 '72103',
 '72105',
 '72107',
 '72109',
 '72111',
 '72113',
 '72115',
 '72117',
 '72119',
 '72121',
 '72123',
 '72125',
 '72127',
 '72129',
 '72131',
 '72133',
 '72135',
 '72137',
 '72139',
 '72141',
 '72143',
 '72145',
 '72147',
 '72149',
 '72151',
 '72153']